In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
df = pd.read_csv("train_5500.csv", sep=":", error_bad_lines=False, names=['trec_6', 'text'])
df.head()

,trec_6,text
0,DESC,manner How did serfdom develop in and then lea...
1,ENTY,cremat What films featured the character Popey...
2,DESC,manner How can I find a list of celebrities ' ...
3,ENTY,animal What fowl grabs the spotlight after the...
4,ABBR,exp What is the full form of .com ?


In [6]:
df['labels_v'] = df.text.str.split(expand=True)[0]

In [20]:
df.text.str.split(expand=True).head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,manner,How,did,serfdom,develop,in,and,then,leave,Russia,...,None,None,None,None,None,None,None,None,None,None
1,cremat,What,films,featured,the,character,Popeye,Doyle,?,None,...,None,None,None,None,None,None,None,None,None,None
2,manner,How,can,I,find,a,list,of,celebrities,',...,None,None,None,None,None,None,None,None,None,None
3,animal,What,fowl,grabs,the,spotlight,after,the,Chinese,Year,...,None,None,None,None,None,None,None,None,None,None
4,exp,What,is,the,full,form,of,.com,?,None,...,None,None,None,None,None,None,None,None,None,None


In [7]:
df['text'] = df.text.str.split(n=1).str[1]

In [8]:
df = df[['trec_6', 'labels_v', 'text']]
display(df.tail())
display(len(df.index))

,trec_6,labels_v,text
5447,ENTY,other,What 's the shape of a camel 's spine ?
5448,ENTY,currency,What type of currency is used in China ?
5449,NUM,temp,What is the temperature today ?
5450,NUM,temp,What is the temperature for cooking ?
5451,ENTY,currency,What currency is used in Australia ?


5452

In [29]:
eval_data = pd.read_csv("TREC_10_label.csv", sep=":", error_bad_lines=False, names=['trec_6', 'text'])
eval_data.head()

,trec_6,labels_v,text
5447,ENTY,other,What 's the shape of a camel 's spine ?
5448,ENTY,currency,What type of currency is used in China ?
5449,NUM,temp,What is the temperature today ?
5450,NUM,temp,What is the temperature for cooking ?
5451,ENTY,currency,What currency is used in Australia ?


5452

In [31]:
eval_data['labels_v'] = df.text.str.split(expand=True)[0]
eval_data['text'] = eval_data.text.str.split(n=1).str[1]
eval_data = eval_data[['trec_6', 'labels_v', 'text']]
display(eval_data.tail())
display(len(eval_data.index))

,trec_6,labels_v,text
495,HUM,What,Who was the 22nd President of the US ?
496,ENTY,Why,What is the money they use in Zambia ?
497,NUM,What,How many feet in a mile ?
498,ENTY,What,What is the birthstone of October ?
499,DESC,Which,What is e-coli ?


500

In [9]:
label_df = df['labels_v'].value_counts().rename_axis('labels_v').reset_index(name='trec50_labels_counts')

label_df['labels'] = label_df.index

label_df.head()

label_dict = dict(zip(label_df.labels_v, label_df.labels))
# 
df['labels'] = df['labels_v'].map(label_dict)
display(df.head())
display(len(df.index))

,trec_6,labels_v,text,labels
0,DESC,manner,How did serfdom develop in and then leave Russ...,5
1,ENTY,cremat,What films featured the character Popeye Doyle ?,7
2,DESC,manner,How can I find a list of celebrities ' real na...,5
3,ENTY,animal,What fowl grabs the spotlight after the Chines...,12
4,ABBR,exp,What is the full form of .com ?,18


5452

In [37]:
# label_df = eval_data['labels_v'].value_counts().rename_axis('labels_v').reset_index(name='trec50_labels_counts')

# label_df['labels'] = label_df.index

# label_df.head()

# label_dict = dict(zip(label_df.labels_v, label_df.labels))
# 
eval_data['labels'] = eval_data['labels_v'].map(label_dict)
display(eval_data.head())
display(len(eval_data.index))

,trec_6,labels_v,text,labels
0,NUM,How,How far is it from Denver to Aspen ?,NaN
1,LOC,What,"What county is Modesto , California in ?",NaN
2,HUM,How,Who was Galileo ?,NaN
3,DESC,What,What is an atom ?,NaN
4,NUM,What,When did Hawaii become a state ?,NaN


500

In [40]:
# eval_data['labels_v'].value_counts()

In [44]:
from sklearn.model_selection import train_test_split
import sklearn

train_data, eval_data = train_test_split(df, test_size=0.1, random_state=123, shuffle=True)

# Configure the model
model_args = ClassificationArgs()
model_args.manual_seed = 123
model_args.num_train_epochs = 1
model_args.train_batch_size = 32
# model_args.evaluate_during_training = True
model_args.max_seq_length = 128
model_args.learning_rate = 4e-5
model_args.eval_batch_size = 16
model_args.weight_decay = 1e-4
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
# model_args.evaluate_during_training = True
# model_args.evaluate_each_epoch = True
# model_args.save_model_every_epoch = True
model_args.save_optimizer_and_scheduler = True
model = ClassificationModel( "xlnet", "xlnet-base-cased", num_labels = 50, args=model_args, use_cuda=False)


# Train the model
model.train_model(train_df = train_data, acc=sklearn.metrics.accuracy_score)

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(eval_df = eval_data, acc=sklearn.metrics.accuracy_score)

 65.43s/it]
Epoch 1 of 1: 100%|██████████| 1/1 [2:46:30<00:00, 9990.60s/it]
INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
  0%|          | 2/546 [00:00<01:02,  8.74it/s]
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlnet_128_50_2
Running Evaluation: 100%|██████████| 35/35 [06:12<00:00, 10.64s/it]
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7000285201662858, 'acc': 0.7234432234432234, 'eval_loss': 1.1730737882001059}


In [11]:
from sklearn.model_selection import train_test_split
import sklearn

train_data, eval_data = train_test_split(df, test_size=0.1, random_state=123, shuffle=True)

# Configure the model
model_args = ClassificationArgs()
model_args.manual_seed = 123
model_args.num_train_epochs = 1
model_args.train_batch_size = 32
# model_args.evaluate_during_training = True
model_args.max_seq_length = 128
model_args.learning_rate = 4e-5
model_args.eval_batch_size = 16
model_args.weight_decay = 1e-4
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
# model_args.evaluate_during_training = True
# model_args.evaluate_each_epoch = True
# model_args.save_model_every_epoch = True
model_args.save_optimizer_and_scheduler = True
model = ClassificationModel( "xlnet", "./outputs", num_labels = 50, args=model_args, use_cuda=False)


# # Train the model
# model.train_model(train_df = train_data, acc=sklearn.metrics.accuracy_score)

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(eval_df = eval_data, acc=sklearn.metrics.accuracy_score)

# Make predictions with the model
predictions, raw_outputs = model.predict(["What type of currency is used in China ?"])

print(predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
  0%|          | 2/546 [00:00<01:38,  5.52it/s]
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlnet_128_50_2
Running Evaluation: 100%|██████████| 35/35 [06:09<00:00, 10.55s/it]
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6759841607099253, 'acc': 0.7014652014652014, 'eval_loss': 1.2545148278985705}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
100%|██████████| 1/1 [00:00<00:00,  1.38it/s][1]

